In [1]:
import csv
import os
import itertools as itt
import classes as vp
import PBI
import parser
import simulator as sim
import plotly.graph_objects as go
import plot
import mathHelper as mh


In [2]:
#simple impirical voting power
#count how often a party in pivotal
#uses votes
def impVotingPower1(votesList, memList):
    pivotal = dict()
    for f in memList:
        count = 0
        for v in votesList:
            if v.isPivotal(memList[f]):
                count += 1
        pivotal[vp.faction_names.get(f,f)] = count
        
    n = sum(pivotal.values())
    
    for p in pivotal:
        pivotal[p] /= n
    
    return pivotal

In [3]:
# find the wining coelition for all the votes
# needs member dict
def allWinningC(data, ids=False):
    coelitions = dict()
    
    for i in range(1, len(vp.party_names) + 1):
        for p in itt.combinations(vp.party_names, i):
            key = list(p)
            key.sort(key=lambda x: vp.party_order[x])
            if ids:
                coelitions[', '.join(key)] = []
            else:
                coelitions[','.join(key)] = 0
    
    for i, v in enumerate(data.votes):
        try:
            #change to lists vot ids instead
            if ids:
                coelitions[', '.join(winningC(data, v))].append(i)
            else:
                coelitions[','.join(winningC(data, v))] += 1
        except KeyError:
            print(v)
            print(i)
        
    return coelitions
    
    
#returns winning coelition for a vote
def winningC(data, vote):
    profiles = dict()
    for f in vp.party_names:
        profiles[f] = vp.VoteProfile()
    
    for mem in vote.yes:
        profiles[vp.faction_names[data.members[mem].faction]].yes += 1
        
    for mem in vote.no:
        profiles[vp.faction_names[data.members[mem].faction]].no += 1
        
    for mem in vote.abstain:
        profiles[vp.faction_names[data.members[mem].faction]].abstain += 1
        
    for mem in vote.novote:
        profiles[vp.faction_names[data.members[mem].faction]].novote += 1
        
    out = list()
    
    for p in profiles:
        if profiles[p].partyVote(abstain=False) == vote.decision:
            out.append(p)
    
    out.sort(key=lambda x: vp.party_order[x])
    return out
        
    

In [4]:
def profie(vote):
    profiles = dict()
    for f in vp.party_names:
        profiles[f] = vp.VoteProfile()
    
    for mem in vote.yes:
        profiles[vp.faction_names[members[mem].faction]].yes += 1
        
    for mem in vote.no:
        profiles[vp.faction_names[members[mem].faction]].no += 1
        
    for mem in vote.abstain:
        profiles[vp.faction_names[members[mem].faction]].abstain += 1
        
    for mem in vote.novote:
        profiles[vp.faction_names[members[mem].faction]].novote += 1
        
    for p in profiles:
        print(p + ' ' + str(profiles[p]))

In [5]:
leg_49 = set(range(20))
leg_50 = set(range(20,39))

sessions = parser.loadSessions()
data = parser.parseFiles(sessions, toParse=leg_50)
factions_size_49 = parser.loadFactionsSize(period='49')
factions_size_50 = parser.loadFactionsSize(period='50')


-------------------

In [6]:
simData = sim.genData(factions_size_50, 4000)

In [63]:
#displays a histogram of all winning coelitions
def coelitions(simpleName = False, normalize=False, cut=0, ele=0):
    l = list(filter(lambda x: x[1] > cut, allWinningC(data).items()))
    l.sort(key=lambda x: x[1], reverse=True)
    
    x = []
    y = []
    
    for i in l:
        if simpleName and (i[0] in coelition_names):
            x.append(coelition_names[i[0]])
        else:
            x.append(i[0])
        y.append(i[1])
    
    if normalize:
        #normalize data
        mh.norm(y, percet=True)
    
    print(sum(y))
    
    if ele != 0:
        x = x[:ele]
        y = y[:ele]
        
    #print(y[:40])
    print(sum(y[:ele]))
    
    
    fig = go.Figure([go.Bar(x=x, y=y)])
    fig.update_layout(#title='Average High and Low Temperatures in New York',
                   #xaxis_title='winning Coalitions',
                   yaxis_title='Occurrence in %')
    fig.show()

#simpler names for coelitions
coelition_names = {'SP,GSP,GLP,CVP,BDP,FDP': 'against SVP',
                  'GLP,CVP,BDP,FDP,SVP': 'against left',
                  'SP,GSP,GLP,CVP,BDP,FDP,SVP': 'unanimous',
                  'CVP,BDP,FDP,SVP': 'center right',
                  'SP,GSP,GLP,CVP,BDP': 'center left',
                  'CVP, FDP, SVP': 'right and CVP',
                  'BDP, FDP, SVP': 'right and BDP',
                  'SP, GSP, GLP, BDP, FDP': 'center left'}
coelitions(normalize=True, cut=0, ele=40, simpleName=False)

100.0000000000002
97.8728168383341


In [30]:
['a' for i in range(3)]

['a', 'a', 'a']

In [46]:
animals=[1, 2, 3]

factions_size = factions_size_50
y_size = 3

fig = go.Figure(data=[
    go.Bar(name='SVP', x=animals, y=[factions_size['SVP'] for i in range(y_size)], marker_color=plot.faction_color[0]),
    go.Bar(name='SP', x=animals, y=[factions_size['SP'] for i in range(y_size)], marker_color=plot.faction_color[1]),
    go.Bar(name='FDP', x=animals, y=[factions_size['FDP'] for i in range(y_size)], marker_color=plot.faction_color[2]),
    go.Bar(name='CVP', x=animals, y=[factions_size['CVP'] for i in range(y_size)], marker_color=plot.faction_color[3]),
    go.Bar(name='GSP', x=animals, y=[factions_size['GSP'] for i in range(y_size)], marker_color=plot.faction_color[4]),
    go.Bar(name='BDP', x=animals, y=[factions_size['BDP'] for i in range(y_size)], marker_color=plot.faction_color[5]),
    go.Bar(name='GLP', x=animals, y=[factions_size['GLP'] for i in range(y_size)], marker_color=plot.faction_color[6])
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [53]:
fig = go.Figure()

parties = list(factions_size_50.keys())
width = list(factions_size_50.values())
width = [x/sum(width) for x in width]
x = [5 for i in range(len(factions_size_50))]

fig.add_trace(go.Bar(
    y=parties,
    x=x,
    width=width,
    name='SF Zoo',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=parties,
    x=x,
    width = width,
    name='LA Zoo',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))

fig.update_layout(barmode='stack')
fig.show()

In [54]:
def showVoteDist(votes):
    #histogramm of how many members votes yes, no, abstain per vote
    present = []
    absent = []
    for v in votes:
        s = len(v.yes) + len(v.no) + len(v.abstain)
        present.append(s)
        absent.append(len(v.novote))
    print('present memers')
    plot.hist(present)
    print('absent members')
    plot.hist(absent)
showVoteDist(data.votes)

present memers


absent members


In [9]:
# creates bar chart with member partisipation rates per faction in votes
def memberPartisipation(data):
    partisipation = dict()
    memPart = vp.KeyList()
    
    def update(key, i, d=partisipation):
        
        key = vp.faction_names[data.members[key].faction]
        
        if not key in d:
            d[key] = [0,0,0,0]
        else:
            d[key][i] += 1
    
    def presents(x):
        return sum(x[0:3]) / sum(x)
    
    
    for v in data.votes:
        for mem in v.yes:
            update(mem, 0)
        for mem in v.no:
            update(mem, 1)
        for mem in v.abstain:
            update(mem, 2)
        for mem in v.novote:
            update(mem, 3)  
    
    
    
    plot.bar_dict(partisipation, value=lambda x: presents(x) * 100, relativ= True, sort= True)
memberPartisipation(data)

In [10]:
# creates bar chart for party unity in votes
def factionUnity(data):
    factionUnity = dict()
    for f in data.factionList:
        factionUnity[vp.faction_names[f]] = []
        
    for v in data.votes:
        for f in data.factionList:
            factionUnity[vp.faction_names[f]].append(v.unity(data.factionList[f], abstain=True))
    plot.bar_dict(factionUnity, value= lambda x: sum(x) / len(x) * 100, relativ=False, sort=True)

In [11]:
factionUnity(data)

In [13]:
def groupedBar_Dict(dicts, names, log=False):
    data = []
    
    for k, d in enumerate(dicts):
        l = []
        for i in d:
            l.append(d[i] * 100)
        
        #for i,item in enumerate(l):
         #   l[i] /= sum(l)
        #norm(l)
        data.append(go.Bar(name=names[k], x=list(d.keys()), y=l))#, marker_color=plot.faction_color))

    fig = go.Figure(data=data)
    # Change the bar mode
    if log:
        fig.update_layout(barmode='group', yaxis_type='log')
    else:
        fig.update_layout(barmode='group')
    fig.show()
    
groupedBar_Dict([impVotingPower1(simData.votes, simData.factionList), impVotingPower1(data.votes, data.factionList), PBI.votingPower(factions_size_50)], ['sim','imp','50'])

In [ ]:
plot.bar_dict(PBI.votingPower(factions_size_50))

In [ ]:
print(simFactions)
print(simVotes[0])

In [ ]:
vp.faction_names

In [ ]:
a = [1,2]
b = ['b', 'a']

a = list(map(lambda x: x *100, a))
print(a)

In [ ]:
ex = {'a' : 3, 'b' : 2, 'c' : 1}
ex2 = {'a' : 4, 'b': 3, 'c': 2, 'd': 1}

In [ ]:
PBI.votingPower(ex, t=4, v=True)


In [ ]:
list(range(1,4))